In [77]:
#This is a tictactoe game 
#People can play this game with AI
#The AI is base on the mlp regressor

import numpy as np
import pickle
import itertools


In [78]:
def startPage():
    print("Welcome!!!")
    print("This is a Connect Four game within 6x7 board")
    print("People can play this game with AI")
    print("The AI is base on the mlp regressor")
    

In [79]:
def generateBoard(row, col):
    board = [['_' for i in range(col)] for i in range(row)]
    print(np.array(board))
    
    #board[5][6]="8"
    #print(np.array(board))
    
    #return board

In [80]:
generateBoard(6,7)

[['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']]


In [81]:
def setPiece(board, role, row, col):
    active = True
    if board[row][col] != "_":
        print("This position has piece already, pls input another one")
        return board, True
    elif role == "Human":
        board[row][col] = "X"
    else:
        board[row][col] = "0"
    
    print([0, 1, 2, 3, 4, 5, 7])
    for count, row in enumerate(board):
        print(count, row)
    
    return board, False
    

In [82]:
def getInput():
    active = True
    while active:
        user_input = input("Do you wana go first? y/n: ")
        if user_input.upper() == "Y":
            active = False
            return user_input
        elif user_input.upper() == "N":
            active = False
            return user_input
        else:
            print("Pls input correctly")

In [83]:
def checkFull(board):
    for i in range(6):
        for j in range(7):
            if board[i][j] == "_":
                return False
    return True
    
    

In [84]:
def checkWinner(board, current):
    
    #check horizental
    for i,row in enumerate(board):
        for j in range(4):
            if board[i][j]==board[i][j+1]==board[i][j+2]==board[i][j+3] and board[i][j] != "_":
                print(current + " is winner on row " + str(i))
                return True
            
    #check vertical
    for col in range(len(board)+1):
        for j in range(3):
            #print(j)
            #print(col)
            if board[j][col]==board[j+1][col]==board[j+2][col]==board[j+3][col] and board[j][col] != "_":
                print(current + " is winner on col " + str(col))
                return True
                  
    #check diagonal
    for i in range(3):
        for j in range(4):
            if (board[i][j]==board[i+1][j+1]==board[i+2][j+2]==board[i+3][j+3] and board[i][j]!= "_") or (board[i][j+3]==board[i+1][j+2]==board[i+2][j+1]==board[i+3][j] and board[i][j+3]!= "_"):
                print(current + " is winner on diagonal ")
                return True
    
    

In [85]:
def getHumanOption(board, boardIndex, current):
    active = True
    while active:
        row_input = int(input("0-5, which row u wana set"))
        col_input = int(input("0-6, which col u wana set"))
        
        if row_input < 0 or row_input > 5:
            print("pls input correct row value")
        elif col_input < 0 or col_input >6:
            print("pls input correct col value")
        else:
            print("="*20)
            boardIndex[6*row_input + col_input] = True
            #if set correctly, run = True, then jump out while loop
            board, run = setPiece(board, current, row_input, col_input)
            active = False
    return board, run

In [86]:
def getAIOption(board, boardIndex, current, mlp_ttt):
    AI_board = np.zeros(9)
    optionTmp = mlp_ttt.predict(AI_board.reshape(1, -1))
    #print(option)
    #print(option[1])
    #print(np.tile(c,(4,2)))
    option = np.tile(optionTmp,(1,5))
    
    for i, idx in enumerate(boardIndex):
        option[0][i] = -10 if (True == idx) else option[0][i]
    tmp = np.amax(option)
    option = np.where(option == tmp, 1, 0)

    ai_move_idx = np.argwhere(option == 1)
    ai_move_idx = ai_move_idx[0][1]

    ai_move_row = ai_move_idx // 6
    ai_move_col = ai_move_idx % 6
            
    boardIndex[ai_move_idx] = True
    board, run = setPiece(board, current, ai_move_row, ai_move_col)
    return board, run

In [87]:
def swtichTurn(board, mlp_ttt, nextToken):
    boardIndex = [False for i in range(6 * 7)]
   
    state = True
    while state:
    #get who is current player
        current = next(nextToken)
        print(current + " turn!")
        run = True
        
        while run:
            if current == "Human":               
                board, run = getHumanOption(board, boardIndex, current)
                #print(run)
            
            else:
                board, run = getAIOption(board, boardIndex, current, mlp_ttt)
                
            
            #if all position is full, state = False, jump out while loop
            end = checkWinner(board, current)
            if end:
                print("game over")
                state = False
            
            if checkFull(board):
                print("All blocks are occupied, no one win in this time!")
                state = False

In [88]:
def runGame():
    startPage()
    
    user_input = getInput()

    role = ["AI"]
    if user_input.upper() == 'Y':
        role.insert(0,"Human")
    else:
        role.append("Human")
    print(role)

    
    print(role[0] + " go first!")
    nextToken = itertools.cycle(role)
    
    
    #generate first board
    row = 6
    col = 7
    generateBoard(row, col)
    board = [["_" for i in range(7)] for i in range(6)]
    
    # load pickle file
    with open('mlp_model.pkl', 'rb') as f:
        mlp_ttt = pickle.load(f)
   

    swtichTurn(board, mlp_ttt, nextToken)
    
    
    
    
runGame()    

Welcome!!!
This is a Connect Four game within 6x7 board
People can play this game with AI
The AI is base on the mlp regressor
Do you wana go first? y/n: y
['Human', 'AI']
Human go first!
[['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_']]
Human turn!
0-5, which row u wana set5
0-6, which col u wana set6
[0, 1, 2, 3, 4, 5, 7]
0 ['_', '_', '_', '_', '_', '_', '_']
1 ['_', '_', '_', '_', '_', '_', '_']
2 ['_', '_', '_', '_', '_', '_', '_']
3 ['_', '_', '_', '_', '_', '_', '_']
4 ['_', '_', '_', '_', '_', '_', '_']
5 ['_', '_', '_', '_', '_', '_', 'X']
AI turn!
[0, 1, 2, 3, 4, 5, 7]
0 ['_', '_', '0', '_', '_', '_', '_']
1 ['_', '_', '_', '_', '_', '_', '_']
2 ['_', '_', '_', '_', '_', '_', '_']
3 ['_', '_', '_', '_', '_', '_', '_']
4 ['_', '_', '_', '_', '_', '_', '_']
5 ['_', '_', '_', '_', '_', '_', 'X']
Human turn!
0-5, which row u wana set4
0-6, which c